In [39]:
import pennylane as qml
from pennylane import numpy as np

## Issue : 

1. **Encoding the Word**:
   - The word is converted to a binary representation using ASCII values. Each character is represented by 8 bits.
   - The total number of bits used is 8 bits per character, which is 24 bits for the word "cab".

2. **Creating the Quantum Circuit**:
   - The binary representation of the word is processed in steps of 3 bits.
   - Each step of 3 bits corresponds to a unique gate operation (X, Y, Z, Hadamard, S, T, or CZ).
   - The total number of bits used is 3 bits per character, which is 9 bits for the word "cab".

3. **Executing the Circuit**:
   - The quantum circuit is executed to generate probabilities for each qubit.
   - The total number of bits used is the same as the number of qubits, which is 9 bits for the word "cab".

4. **Decoding the Word**:
   - The probabilities are processed in steps of 3 bits to determine the most likely outcome for each qubit.
   - Each step of 3 bits corresponds to a character in the original word.
   - The total number of bits used is 3 bits per character, which is 9 bits for the word "cab".

5. **Decoding the Word (Function)**:
   - The function processes the probabilities in steps of 3 bits to determine the most likely outcome for each qubit.
   - Each step of 3 bits corresponds to a character in the original word.
   - The total number of bits used is 3 bits per character, which is 9 bits for the word "cab".

In summary, the number of bits used at each step is:

- Encoding: 24 bits
- Creating the Quantum Circuit: 9 bits
- Executing the Circuit: 9 bits
- Decoding the Word: 9 bits
- Decoding the Word (Function): 9 bits

Issue faced is bit conversion - Let's check


In [57]:
# Lookup table for 3-bit encoding
encoding = {
    'a': '001',
    'b': '010',
    'c': '011',
    'd': '100',
    'e': '101',
    'f': '110',
    'g': '111',
    'h': '000',
    'i': '101',
    'j': '110',
    'k': '111',
    'l': '000',
    'm': '001',
    'n': '010',
    'o': '011',
    'p': '100',
    'q': '101',
    'r': '110',
    's': '111',
    't': '000',
    'u': '001',
    'v': '010',
    'w': '011',
    'x': '100',
    'y': '101',
    'z': '110'
}

In [58]:
def encode(input_string) : 
    output_string = ''
    for char in input_string.lower() : 
        output_string += encoding.get(char, '')

    return output_string


Example word with three bits per character 

In [59]:
word = "cab" # Example word to encode

Convert the word into a binary representation


In [60]:
binary_word = encode(word)

Create the quantum circuit

In [61]:
dev = qml.device("default.qubit", wires = len(binary_word))

@qml.qnode(dev)
def circuit(binary_word):
    for i in range(0, len(binary_word), 3) : 
        bit_sequence = binary_word[i:i+3]

        # Convert the three-bit sequence to an integer between 0 and 7
        value = int(bit_sequence, 2)

    # Use the integer valye to determine which gates to apply
    if value == 0: 
        qml.PauliX(i)   # Apply X gate for '000'

    elif value == 1:
        qml.PauliX(i)   # Apply X gate for '001'

    elif value == 2:
            qml.PauliY(i)  # Apply Y gate for '010'
        
    elif value == 3:
        qml.PauliZ(i)  # Apply Z gate for '011'
        
    elif value == 4:
        qml.Hadamard(i)  # Apply Hadamard gate for '100'
    
    elif value == 5:
        qml.S(i)  # Apply S gate for '101'
    
    elif value == 6:
        qml.T(i)  # Apply T gate for '110'
    
    elif value == 7:
        qml.CZ((i, (i+1)%len(binary_qord)))     # Apply CZ gate for '111'

    return [qml.probs(wires = i) for i in range(len(binary_word))]

Execute the circuit to get probabilies


In [62]:
probabilities = circuit(binary_word)


In [64]:
probabilities

[tensor([1., 0.], requires_grad=True),
 tensor([1., 0.], requires_grad=True),
 tensor([1., 0.], requires_grad=True),
 tensor([1., 0.], requires_grad=True),
 tensor([1., 0.], requires_grad=True),
 tensor([1., 0.], requires_grad=True),
 tensor([0., 1.], requires_grad=True),
 tensor([1., 0.], requires_grad=True),
 tensor([1., 0.], requires_grad=True)]

Decode the probabilities to retrieve the original word 

In [66]:
retrieved_word = ''

for i in range(0, len(binary_word), 3):  # Process every three bits
    bit_sequence = binary_word[i:i+3]

    # Convert the three-bit sequence back to its original form
    qubit_probabilities = [p[1] for p in probabilities[i:i+3]]  # Calculate the probabilities for each qubit

    # Determine the most likely outcome for each qubit
    for qubit_probability in qubit_probabilities:
        if qubit_probability > 0.5:
            bit_sequence += '1'
        else:
            bit_sequence += '0'

    retrieved_word += chr(int(bit_sequence, 2))

In [67]:
print(f"Bit Sequence : {bit_sequence}, Probabilities : {qubit_probabilities}, Determined Bits : {bit_sequence}")

Bit Sequence : 010100, Probabilities : [tensor(1., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True)], Determined Bits : 010100


Add a new function to decode the word from the ouput probabilities 


In [68]:
def decode_word(probabilities) : 
    decoded_word = ''
    for i in range(0, len(probabilities), 3):
        qubit_probabilities = [p[1] for p in probabilities[i:i+3]]  # Calculate the probabilities for each qubit

        bit_sequence = ''

        for qubit_probability in qubit_probabilities:
            if qubit_probability > 0.5:
                bit_sequence += '1'
            else:
                bit_sequence += '0'

        decoded_word += chr(int(bit_sequence, 2))

    return decoded_word

In [69]:
probabilities

[tensor([1., 0.], requires_grad=True),
 tensor([1., 0.], requires_grad=True),
 tensor([1., 0.], requires_grad=True),
 tensor([1., 0.], requires_grad=True),
 tensor([1., 0.], requires_grad=True),
 tensor([1., 0.], requires_grad=True),
 tensor([0., 1.], requires_grad=True),
 tensor([1., 0.], requires_grad=True),
 tensor([1., 0.], requires_grad=True)]

Retrieve the original word from the output probabilities 

In [70]:
retrieved_word_probabilities = decode_word(probabilities)

In [71]:
print(f"Encoded word : {word}")

Encoded word : cab


In [72]:
print(f"Retrieved Word : {retrieved_word_probabilities}")

Retrieved Word :   
